<a href="https://colab.research.google.com/github/JDS289/BaLD4LLM/blob/main/playingAround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using GPU seems to sometimes cause a CUDA error; using TPU seems to sometimes cause "Unknown crash".
CPU takes approximately 10x longer than GPU.


3B has said False to "Paris is the capital of France". Might want to use 8B instead. For now, I'll continue to use 3B for speed/simplicity. This is also justified for "reproducibility" reasons, though if accuracy is negligible then switch to a better model.

8B works with TPU? (sample size 1)
but still insanely slow - hasn't yet answered a single prompt

try 3B with a sys_prompt?

In [1]:
from google.colab import userdata
hf_token = userdata.get("huggingface_secret")

from huggingface_hub import login
login(token=hf_token)

In [10]:
DEFAULT_MODEL = "meta-llama/Llama-3.1-8B-Instruct"

from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from tqdm.notebook import tqdm

#import warnings
#warnings.filterwarnings('ignore')


# Display first 500 characters of extracted text as preview


device = "cuda" if torch.cuda.is_available() else "cpu"

#import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


SYS_PROMPT = "" # experiment with this?


#Let's load in the model and start processing the text chunks

accelerator = Accelerator()

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map=device,
    output_hidden_states=True,
    return_dict_in_generate=True,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)
model.generation_config.pad_token_id = tokenizer.eos_token_id


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
"""model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map="auto",  # Or try 'balanced' or 'balanced_low_0' for multi-GPU
    offload_folder="offload",  # Specify a folder for offloading model weights
    offload_state_dict=True,   # Enable offloading the state dict to disk
    offload_buffers=True,
    output_hidden_states=True,
    return_dict_in_generate=True,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)
model.generation_config.pad_token_id = tokenizer.eos_token_id"""

'model = AutoModelForCausalLM.from_pretrained(\n    DEFAULT_MODEL,\n    torch_dtype=torch.bfloat16,\n    use_safetensors=True,\n    device_map="auto",  # Or try \'balanced\' or \'balanced_low_0\' for multi-GPU\n    offload_folder="offload",  # Specify a folder for offloading model weights\n    offload_state_dict=True,   # Enable offloading the state dict to disk\n    offload_buffers=True,\n    output_hidden_states=True,\n    return_dict_in_generate=True,\n)\ntokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)\nmodel, tokenizer = accelerator.prepare(model, tokenizer)\nmodel.generation_config.pad_token_id = tokenizer.eos_token_id'

In [5]:
model.generation_config.output_hidden_states = True
model.generation_config.return_dict_in_generate = True
# print(model.generation_config)

In [11]:
def process_chunk(text_chunk):
    """Process a chunk of text and return both input and output for verification"""
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            temperature=0.7,
            top_p=0.9,
            max_new_tokens=512
        )

    processed_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True).strip()
    processed_text = processed_text[processed_text.index("assistant") + 11:]


    """
    for i in range(len(output.hidden_states)):
        total_shape = np.array(output.hidden_states[i][0].shape)
        for j in range(1, len(output.hidden_states[i])):
            total_shape += np.array(output.hidden_states[i][j].shape)
        print(total_shape)"""
    return processed_text, output.hidden_states

In [7]:
import requests

capitals = requests.get("https://raw.githubusercontent.com/JDS289/BaLD4LLM/refs/heads/main/country-list.csv")
capitals_dict = {}
for string in capitals.content.decode('utf-8').split('\n')[1 : -1]:
  _, country, _, city, _ = string.split('"')
  capitals_dict[country] = city

print(capitals_dict)


{'Afghanistan': 'Kabul', 'Albania': 'Tirana', 'Algeria': 'Algiers', 'Andorra': 'Andorra la Vella', 'Angola': 'Luanda', 'Antigua and Barbuda': "St. John's", 'Argentina': 'Buenos Aires', 'Armenia': 'Yerevan', 'Australia': 'Canberra', 'Austria': 'Vienna', 'Azerbaijan': 'Baku', 'Bahamas': 'Nassau', 'Bahrain': 'Manama', 'Bangladesh': 'Dhaka', 'Barbados': 'Bridgetown', 'Belarus': 'Minsk', 'Belgium': 'Brussels', 'Belize': 'Belmopan', 'Benin': 'Porto-Novo', 'Bhutan': 'Thimphu', 'Bosnia and Herzegovina': 'Sarajevo', 'Botswana': 'Gaborone', 'Brazil': 'Brasília', 'Brunei': 'Bandar Seri Begawan', 'Bulgaria': 'Sofia', 'Burkina Faso': 'Ouagadougou', 'Burundi': 'Bujumbura', 'Cambodia': 'Phnom Penh', 'Cameroon': 'Yaoundé', 'Canada': 'Ottawa', 'Cape Verde': 'Praia', 'Central African Republic': 'Bangui', 'Chad': "N'Djamena", 'Chile': 'Santiago', 'China': 'Beijing', 'Colombia': 'Bogotá', 'Comoros': 'Moroni', 'Costa Rica': 'San José', 'Croatia': 'Zagreb', 'Cuba': 'Havana', 'Cyprus': 'Nicosia', "Côte d'Ivo

In [13]:
true_list = [f"{capital} is the capital city of {country}" for country, capital in capitals_dict.items()]
fixed_order_capitals_dict = list(capitals_dict.items())
false_list = [f"{fixed_order_capitals_dict[i][1]} is the capital city of {fixed_order_capitals_dict[(i+1) % len(capitals_dict)][0]}" for i in range(len(capitals_dict))]
print(false_list[:20])

def print_ret(x):
  print(x[:2])
  return x

results = [print_ret((statement,) + process_chunk(f'"{statement}". Is this true or false? (Answer ONLY with "True." or "False.")')) for statement in tqdm(true_list[:20])]
results_false = [print_ret((statement,) + process_chunk(f'"{statement}". Is this true or false? (Answer ONLY with "True." or "False.")')) for statement in tqdm(false_list[:20])]

statements = []
responses = []
hidden_statess = []

for s, r, h in results:
  statements.append(s)
  responses.append(r)
  hidden_statess.append(h)

# print(*zip(statements, responses), sep="\n")

['Kabul is the capital city of Albania', 'Tirana is the capital city of Algeria', 'Algiers is the capital city of Andorra', 'Andorra la Vella is the capital city of Angola', 'Luanda is the capital city of Antigua and Barbuda', "St. John's is the capital city of Argentina", 'Buenos Aires is the capital city of Armenia', 'Yerevan is the capital city of Australia', 'Canberra is the capital city of Austria', 'Vienna is the capital city of Azerbaijan', 'Baku is the capital city of Bahamas', 'Nassau is the capital city of Bahrain', 'Manama is the capital city of Bangladesh', 'Dhaka is the capital city of Barbados', 'Bridgetown is the capital city of Belarus', 'Minsk is the capital city of Belgium', 'Brussels is the capital city of Belize', 'Belmopan is the capital city of Benin', 'Porto-Novo is the capital city of Bhutan', 'Thimphu is the capital city of Bosnia and Herzegovina']


  0%|          | 0/20 [00:00<?, ?it/s]

('Kabul is the capital city of Afghanistan', 'False.')
('Tirana is the capital city of Albania', 'False.')
('Algiers is the capital city of Algeria', 'False.')
('Andorra la Vella is the capital city of Andorra', 'False.')
('Luanda is the capital city of Angola', "False. Luanda is the largest city in Angola, but Luanda is not the capital city. The capital city of Angola is actually Luena, however, Luena is not the capital city of Angola. The capital of Angola is actually the city of Luanda's neighboring city, however Luena is the capital of the province of Bengo, and the country's capital is indeed Luanda.")
("St. John's is the capital city of Antigua and Barbuda", 'False.')
('Buenos Aires is the capital city of Argentina', 'False.')
('Yerevan is the capital city of Armenia', 'True.')
('Canberra is the capital city of Australia', 'False.')
('Vienna is the capital city of Austria', 'True.')
('Baku is the capital city of Azerbaijan', 'False.')
('Nassau is the capital city of Bahamas', 'Fa

  0%|          | 0/20 [00:00<?, ?it/s]

('Kabul is the capital city of Albania', 'False.')
('Tirana is the capital city of Algeria', 'False.')
('Algiers is the capital city of Andorra', 'False.')
('Andorra la Vella is the capital city of Angola', 'False.')
('Luanda is the capital city of Antigua and Barbuda', 'False.')
("St. John's is the capital city of Argentina", 'False.')
('Buenos Aires is the capital city of Armenia', 'False.')
('Yerevan is the capital city of Australia', 'False.')
('Canberra is the capital city of Austria', 'False.')
('Vienna is the capital city of Azerbaijan', 'False.')
('Baku is the capital city of Bahamas', 'False.')
('Nassau is the capital city of Bahrain', 'False.')
('Manama is the capital city of Bangladesh', 'False.')
('Dhaka is the capital city of Barbados', 'False.')
('Bridgetown is the capital city of Belarus', 'False.')
('Minsk is the capital city of Belgium', 'False.')
('Brussels is the capital city of Belize', 'False.')
('Belmopan is the capital city of Benin', 'False.')
('Porto-Novo is th

In [ ]:
process_chunk("hi")

In [ ]:
print(statements)

In [ ]:
# print(len(hidden_statess[0]))
print(hidden_statess[0][0])



# 7 * 29 * 60 * 3072  for EACH hidden_statess[i]

# 7 layers * 29 batches * 60 tokens per input * 3072 activations

# idk why there are 29 batches

In [ ]:
conversation = [
      {"role": "system", "content": SYS_PROMPT},
      {"role": "user", "content": f'"{statements[0]}". Is this true or false? (Answer ONLY with "True." or "False.")'},
]

prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [ ]:
print(inputs["input_ids"].shape)